# Training our first Convolutional Neural Network

In this notebook we are going to build and train our first Convolutional Neural Network (CNN). We will also familiarize with the usage of Batch Normalization (BN) layers, and see it in action on [LeNet](https://ieeexplore.ieee.org/document/726791).

## Batch Normalization

The aim of **batch normalization layers** is to standardize features within a mini-batch, such that their mean is 0 and variance is 1.

More details can be found in the original [paper](https://arxiv.org/abs/1502.03167).

$BN(x_{i, k}) = \gamma_{k} \frac{x_{i, k} - \mu_{B, k}}{\sqrt{\sigma^{2}_{B,k} + \epsilon}} + \beta_{k}$

With $\gamma_{k}$ and $\beta_{k}$ being learnable parameters.

The intuitive idea behind BN is as follows: a neural network is trained using mini-batches, and the distribution of inputs **varies** from one batch to the other. Difference in distributions between mini-batches can cause the training to be **unstable** and heavily **dependant on the initial weights** of the network. Therefore, this kind of transformation (transforming the inputs to have mean 0 and unit variance) guarantees that input distribution of each layer remains **unchanged across mini-batches**.

More interestingly, we will learn how to code BN layer from scratch using PyTorch. Let's start by importing the necessary libraries, as usual.

We start, as usual, by importing the necessary libraries.

In [ ]:
%pip install tensorboard

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm # Progress bars!

We will first implement a BN layer for 1D tensors and then expand the implementation to the 2D case.

In [ ]:
"""
Applies Batch Normalization over a 1D input (or 2D tensor)

Shape:
  Input: (N, C)
  Output: (N, C)

Input Parameters:
  in_features: number of features of the input activations
  track_running_stats: whether to keep track of running mean and std. (default: True)
  affine: whether to scale and shift the normalized activations. (default: True)
  momentum: the momentum value for the moving average. (default: 0.9)

Usage:
  >>> # with learable parameters
  >>> bn = BatchNorm1d(4)
  >>> # without learable parameters
  >>> bn = BatchNorm1d(4, affine=False)
  >>> input = torch.rand(10, 4)
  >>> out = bn(input)
"""

class BatchNorm1d(nn.Module):
    def __init__(self, in_features, track_running_stats=True, affine=True, momentum=0.9):
        super().__init__()

        self.in_features = in_features
        self.track_running_stats = track_running_stats
        self.affine = affine
        self.momentum = momentum

        if self.affine:
            self.gamma = nn.Parameter(torch.ones(self.in_features, 1))
            self.beta = nn.Parameter(torch.zeros(self.in_features, 1))

        if self.track_running_stats:
            # register_buffer registers a tensor as a buffer that will be saved as part of the model
            # but which does not require to be trained, differently from nn.Parameter
            self.register_buffer('running_mean', torch.zeros(self.in_features, 1))
            self.register_buffer('running_std', torch.ones(self.in_features, 1))

    def forward(self, x):
        # Transpose (N, C) to (C, N)
        x = x.transpose(0, 1).contiguous().view(x.shape[1], -1)

        # Calculate batch mean
        mean = x.mean(dim=1).view(-1, 1)

        # Calculate batch std
        std = x.std(dim=1).view(-1, 1)

        # During training keep running statistics (moving average of mean and std)
        if self.training and self.track_running_stats:
            # No computational graph is necessary to be built for this computation
            with torch.no_grad():
                self.running_mean = self.momentum * self.running_mean + (1 - self.momentum) * mean
                self.running_std = self.momentum * self.running_std + (1 - self.momentum) * std

        # During inference time
        if not self.training and self.track_running_stats:
            mean = self.running_mean
            std = self.running_std

        # Normalize the input activations
        x = (x - mean) / std

        # Scale and shift the normalized activations
        if self.affine:
            x = x * self.gamma + self.beta

        # Transpose back to original shape
        return x.transpose(0, 1)

"""
Applies Batch Normalization over a 2D or 3D input (4D tensor)

Shape:
  Input: (N, C, H, W)
  Output: (N, C, H, W)

Input Parameters:
  in_features: number of features of the input activations
  track_running_stats: whether to keep track of running mean and std. (default: True)
  affine: whether to scale and shift the normalized activations. (default: True)
  momentum: the momentum value for the moving average. (default: 0.9)

Usage:
  >>> # with learable parameters
  >>> bn = BatchNorm2d(4)
  >>> # without learable parameters
  >>> bn = BatchNorm2d(4, affine=False)
  >>> input = torch.rand(10, 4, 5, 5)
  >>> out = bn(input)
"""

class BatchNorm2d(nn.Module):
    def __init__(self, in_features, track_running_stats=True, affine=True, momentum=0.9):
        super().__init__()

        self.in_features = in_features
        self.track_running_stats = track_running_stats
        self.affine = affine
        self.momentum = momentum

        if self.affine:
            self.gamma = nn.Parameter(torch.ones(self.in_features, 1))
            self.beta = nn.Parameter(torch.zeros(self.in_features, 1))

        if self.track_running_stats:
            # register_buffer registers a tensor as a buffer that will be saved as part of the model
            # but which does not require to be trained, differently from nn.Parameter
            self.register_buffer('running_mean', torch.zeros(self.in_features, 1))
            self.register_buffer('running_std', torch.ones(self.in_features, 1))

    def forward(self, x):
        # Transpose (N, C, H, W) to (C, N, H, W)
        x = x.transpose(0, 1)

        # Store the shape
        c, bs, h, w = x.shape

        # Collapse all dimensions except the 'channel' dimension
        x = x.contiguous().view(c, -1)

        # Calculate batch mean
        mean = x.mean(dim=1).view(-1, 1)

        # Calculate batch std
        std = x.std(dim=1).view(-1, 1)

        # Keep running statistics (moving average of mean and std)
        if self.training and self.track_running_stats:
            with torch.no_grad():
                self.running_mean = self.momentum * self.running_mean + (1 - self.momentum) * mean
                self.running_std = self.momentum * self.running_std + (1 - self.momentum) * std

        # During inference time
        if not self.training and self.track_running_stats:
            mean = self.running_mean
            std = self.running_std

        # Normalize the input activations
        x = (x - mean) / std

        # Scale and shift the normalized activations
        if self.affine:
            x = x * self.gamma + self.beta

        return x.view(c, bs, h, w).transpose(0, 1)

## Building our first CNN: LeNet-5

![image.png](attachment:2cafc123-66e4-4713-9b91-e648b72a6388.png)

In order to build this model, we are going to need some **convolutional** and some **fully connected** layers. The former can be easily defined by exploiting the `torch.nn.Conv2D` module from PyTorch. Remember you can always take a look at the [documentation](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)! We will also be using pooling operations (Max Pooling) to reduce the size of the feature maps. In particular, we are going to use the `torch.nn.functional.max_pool2d` module (details can be found, as usual, in the [docs](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html)). Furthermore, for this model we are going to need the Rectified Linear Unit (ReLU) activation, available in the `torch.nn.functional.relu` module (details [here](https://pytorch.org/docs/stable/generated/torch.nn.functional.relu.html#torch.nn.functional.relu)).


In [ ]:
class LeNet(nn.Module):
    def __init__(self, with_bn=False):
        super().__init__()
        self.with_bn = with_bn

        # input channel = 3, output channels = 6, kernel size = 5
        # input image size = (28, 28), image output size = (24, 24)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=(5, 5))
        self.bn1 = BatchNorm2d(6) if self.with_bn else nn.Identity()

        # input channel = 6, output channels = 16, kernel size = 5
        # input image size = (12, 12), output image size = (8, 8)
        self.conv2 = torch.nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5, 5))
        self.bn2 = BatchNorm2d(16) if self.with_bn else nn.Identity()

        # input dim = 5 * 5 * 16 (H x W x C), output dim = 120
        self.fc3 = torch.nn.Linear(in_features=5 * 5 * 16, out_features=120)
        self.bn3 = BatchNorm1d(120) if self.with_bn else nn.Identity()

        # input dim = 120, output dim = 84
        self.fc4 = torch.nn.Linear(in_features=120, out_features=84)
        self.bn4 = BatchNorm1d(84) if self.with_bn else nn.Identity()

        # input dim = 84, output dim = 10
        self.fc5 = torch.nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        # First convolutional layer + relu
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)

        # Max Pooling with kernel size = 2
        # Output size = (12, 12)
        x = F.max_pool2d(x, kernel_size=2)

        # Second convolutional layer + relu
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)

        # Max Pooling with kernel size = 2
        # Output size = (4, 4)
        x = F.max_pool2d(x, kernel_size=2)

        # Flatten the feature maps into a long vector (-> (bs, 4*4*16))
        x = x.view(x.shape[0], -1)

        # First linear layer + relu
        x = self.fc3(x)
        x = self.bn3(x)
        x = F.relu(x)

        # Second linear layer + relu
        x = self.fc4(x)
        x = self.bn4(x)
        x = F.relu(x)

        # Output layer (linear)
        x = self.fc5(x)

        return x

## Optimizer & cost function
We are going to use the familiar [Stochastic Gradient Descent (SGD)](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html) optimizer and the [Cross Entropy Loss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) for our optimization.

In [ ]:
def get_cost_function():
    cost_function = torch.nn.CrossEntropyLoss()
    return cost_function

def get_optimizer(net, lr, wd, momentum):
    optimizer = torch.optim.SGD(net.parameters(), lr=lr, weight_decay=wd, momentum=momentum)
    return optimizer

## Training and test steps
We are going to implement our training and test pipelines as discussed in the previous lab sessions.

In [ ]:
def train_one_epoch(net, data_loader, optimizer, cost_function, device="cuda"):
    samples = 0.0
    cumulative_loss = 0.0
    cumulative_accuracy = 0.0

    # Set the network to training mode
    net.train()

    # Iterate over the training set
    for batch_idx, (inputs, targets) in enumerate(data_loader):
        # Load data into GPU
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Forward pass
        outputs = net(inputs)

        # Loss computation
        loss = cost_function(outputs,targets)

        # Backward pass
        loss.backward()

        # Parameters update
        optimizer.step()

        # Gradients reset
        optimizer.zero_grad()

        # Fetch prediction and loss value
        samples += inputs.shape[0]
        cumulative_loss += loss.item()
        _, predicted = outputs.max(dim=1) # max() returns (maximum_value, index_of_maximum_value)

        # Compute training accuracy
        cumulative_accuracy += predicted.eq(targets).sum().item()

    return cumulative_loss/samples, cumulative_accuracy / samples * 100

def test(net, data_loader, cost_function, device="cuda"):
    samples = 0.0
    cumulative_loss = 0.0
    cumulative_accuracy = 0.0

    # Set the network to evaluation mode
    net.eval()

    # Disable gradient computation (we are only testing, we do not want our model to be modified in this step!)
    with torch.no_grad():
        # Iterate over the test set
        for batch_idx, (inputs, targets) in enumerate(data_loader):

            # Load data into GPU
            inputs = inputs.to(device)
            targets = targets.to(device)

            # Forward pass
            outputs = net(inputs)

            # Loss computation
            loss = cost_function(outputs, targets)

            # Fetch prediction and loss value
            samples+=inputs.shape[0]
            cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
            _, predicted = outputs.max(1)

            # Compute accuracy
            cumulative_accuracy += predicted.eq(targets).sum().item()

    return cumulative_loss / samples, cumulative_accuracy / samples * 100

## Data loading
In this block we are going to define our **data loading** utility. Differently from last time, in this case we are going to introduce **normalization**. This step is needed in order **bound** our values to the `[-1,1]` range, and obtain a **stable** training process for our network. This can be achieved by using the `torchvision.transforms.Normalize()` module (details [here](https://pytorch.org/vision/main/generated/torchvision.transforms.Normalize.html)).

In [ ]:
def get_data(batch_size, test_batch_size=256, dataset='mnist'):
    # Prepare data transformations and then combine them sequentially
    if dataset == 'mnist':
        transform = T.Compose([                                                      # Compose the above transformations into one
            T.ToTensor(),                                                            # Convert Numpy to Pytorch Tensor
            T.Lambda(lambda x: F.pad(x, (2, 2, 2, 2), 'constant', 0)),               # Pad zeros to make MNIST 32 x 32
            T.Lambda(lambda x: x.repeat(3, 1, 1)),                                   # Make MNIST RGB instead of grayscale
            T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])                   # Normalize the Tensors between [-1, 1]
        ])
    elif dataset == 'svhn':
        transform = T.Compose([                                                      # Compose the above transformations into one
            T.ToTensor(),                                                            # Convert Numpy to Pytorch Tensor
            T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])                   # Normalize the Tensors between [-1, 1]
        ])

    # Prepare dataset
    if dataset == 'mnist':
        full_training_data = torchvision.datasets.MNIST('./data/mnist', train=True, transform=transform, download=True)
        test_data = torchvision.datasets.MNIST('./data/mnist', train=False, transform=transform, download=True)
    elif dataset == 'svhn':
        full_training_data = torchvision.datasets.SVHN('./data/svhn', split='train', transform=transform, download=True)
        test_data = torchvision.datasets.SVHN('./data/svhn', split='test', transform=transform, download=True)

    # Create train and validation splits
    num_samples = len(full_training_data)
    training_samples = int(num_samples * 0.8 + 1)
    validation_samples = num_samples - training_samples
    training_data, validation_data = torch.utils.data.random_split(full_training_data, [training_samples, validation_samples])

    # Initialize dataloaders
    train_loader = torch.utils.data.DataLoader(training_data, batch_size, shuffle=True, drop_last=True, num_workers=8)
    val_loader = torch.utils.data.DataLoader(validation_data, test_batch_size, shuffle=False, num_workers=8)
    test_loader = torch.utils.data.DataLoader(test_data, test_batch_size, shuffle=False, num_workers=8)

    return train_loader, val_loader, test_loader

## Put it all together!
We are now ready to combine all the ingredients defined so far into our **training procedure**. We define a main function that **initializes** everything, **trains** the model over multiple epochs and **logs** the results.

In [ ]:
def main(run_name,
         batch_size=128,
         device="cuda",
         learning_rate=0.01,
         weight_decay=0.000001,
         momentum=0.9,
         epochs=25,
         dataset="mnist",
         with_bn=False):
    # Creates a logger for the experiment
    writer = SummaryWriter(log_dir=f"runs/{run_name}")

    # Get dataloaders
    train_loader, val_loader, test_loader = get_data(batch_size=batch_size, test_batch_size=batch_size, dataset=dataset)

    # Instantiate model and send it to cuda device
    net = LeNet(with_bn=with_bn).to(device)

    # Instatiate optimizer and cost function
    optimizer = get_optimizer(net, learning_rate, weight_decay, momentum)
    cost_function = get_cost_function()

    # Run a single test step beforehand and print metrics
    print("Before training:")
    train_loss, train_accuracy = test(net, train_loader, cost_function, device=device)
    val_loss, val_accuracy = test(net, val_loader, cost_function, device=device)
    test_loss, test_accuracy = test(net, test_loader, cost_function, device=device)
    print(f"\tTraining loss {train_loss:.5f}, Training accuracy {train_accuracy:.2f}")
    print(f"\tValidation loss {val_loss:.5f}, Validation accuracy {val_accuracy:.2f}")
    print(f"\tTest loss {test_loss:.5f}, Test accuracy {test_accuracy:.2f}")

    # Add values to plots
    writer.add_scalar("train/loss", train_loss, 0)
    writer.add_scalar("train/accuracy", train_accuracy, 0)
    writer.add_scalar("val/loss", val_loss, 0)
    writer.add_scalar("val/accuracy", val_accuracy, 0)
    writer.add_scalar("test/loss", test_loss, 0)
    writer.add_scalar("test/accuracy", test_accuracy, 0)

    # Iterate over the number of epochs
    pbar = tqdm(range(epochs), desc="Training")
    for e in pbar:
        # Train and log
        train_loss, train_accuracy = train_one_epoch(net, train_loader, optimizer, cost_function, device=device)
        val_loss, val_accuracy = test(net, val_loader, cost_function, device=device)
        # print(f"Epoch: {e + 1:d}")
        # print(f"\tTraining loss {train_loss:.5f}, Training accuracy {train_accuracy:.2f}")
        # print(f"\tVal loss {val_loss:.5f}, Val accuracy {val_accuracy:.2f}")
        # print("-----------------------------------------------------")

        # Add values to plots
        writer.add_scalar("train/loss", train_loss, e + 1)
        writer.add_scalar("train/accuracy", train_accuracy, e + 1)
        writer.add_scalar("val/loss", val_loss, e + 1)
        writer.add_scalar("val/accuracy", val_accuracy, e + 1)

        pbar.set_postfix(train_loss=train_loss, train_accuracy=train_accuracy, val_loss=val_loss, val_accuracy=val_accuracy)

    # Compute and print final metrics
    print("After training:")
    train_loss, train_accuracy = test(net, train_loader, cost_function, device=device)
    val_loss, val_accuracy = test(net, val_loader, cost_function, device=device)
    test_loss, test_accuracy = test(net, test_loader, cost_function, device=device)

    print(f"\tTraining loss {train_loss:.5f}, Training accuracy {train_accuracy:.2f}")
    print(f"\tValidation loss {val_loss:.5f}, Validation accuracy {val_accuracy:.2f}")
    print(f"\tTest loss {test_loss:.5f}, Test accuracy {test_accuracy:.2f}")

    # Add values to plots
    writer.add_scalar("train/loss", train_loss, epochs + 1)
    writer.add_scalar("train/accuracy", train_accuracy, epochs + 1)
    writer.add_scalar("val/loss", val_loss, epochs + 1)
    writer.add_scalar("val/accuracy", val_accuracy, epochs + 1)
    writer.add_scalar("test/loss", test_loss, epochs + 1)
    writer.add_scalar("test/accuracy", test_accuracy, epochs + 1)

    # Close writer
    writer.close()

## Run!

Let's first train on mnist and svhn without BN

In [ ]:
main("mnist", dataset="mnist")

Before training:
	Training loss 0.01799, Training accuracy 11.21
	Validation loss 0.01804, Validation accuracy 11.37
	Test loss 0.01819, Test accuracy 11.35


Training: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:49<00:00,  1.97s/it, train_accuracy=100, train_loss=9.05e-6, val_accuracy=99, val_loss=0.000398]

After training:


	Training loss 0.00000, Training accuracy 99.99
	Validation loss 0.00040, Validation accuracy 98.98
	Test loss 0.00036, Test accuracy 99.02


In [ ]:
main("svhn", dataset="svhn")

Using downloaded and verified file: ./data/svhn/train_32x32.mat
Using downloaded and verified file: ./data/svhn/test_32x32.mat
Before training:
	Training loss 0.01803, Training accuracy 6.31
	Validation loss 0.01812, Validation accuracy 6.50
	Test loss 0.01810, Test accuracy 6.13


Training: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:59<00:00,  2.39s/it, train_accuracy=96.5, train_loss=0.000863, val_accuracy=88.5, val_loss=0.00466]

After training:


	Training loss 0.00078, Training accuracy 96.73
	Validation loss 0.00466, Validation accuracy 88.48
	Test loss 0.00521, Test accuracy 87.00


... and now with BN

In [ ]:
main("mnist_bn", dataset="mnist", with_bn=True)

In [ ]:
main("svhn_bn", dataset="svhn", with_bn=True)

Let's visualize the results with TensorBoard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=runs